In [16]:
import numpy as np
import nashpy as nash
import random
import matplotlib.pyplot as plt

In [82]:
import pygame

In [83]:
class Player:
    def __init__(self, isCarn = None, awareness=None, speed=None,id = -1):
        self.isCarn = isCarn
        self.id = id
        self.health = 100
        self.awareness=0
        self.speed=3
        self.x = 0
        self.y = 0

In [100]:
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

class Simulation:
    def __init__(self, N=75, N_carns=25, N_herbs=50, env_size=250, food_density=0.25, payoff_matrix=None):
        self.N = N
        self.N_carns = N_carns
        self.N_herbs = N_herbs
        self.carns = []
        self.herbs = []
        self.food_density = food_density
        self.env_size = env_size
        self.environment = np.full((env_size, env_size), 0)
        self.carn_loc = np.full((env_size, env_size), None)
        self.herb_loc = np.full((env_size, env_size), None)
        self.payoff_matrix = payoff_matrix

    def spawnPlayers(self):
        id = 1
        for i in range(self.N_carns):
            carn = Player(True, id)
            self.carns.append(carn)
            id += 1

        for i in range(self.N_herbs):
            herb = Player(False, id)
            self.herbs.append(herb)
            id += 1

    def resetPlayerPositions(self):
        edges = {
            'top': [[0, i] for i in range(self.env_size)],
            'bottom': [[self.env_size - 1, i] for i in range(self.env_size)],
            'left': [[i, 0] for i in range(self.env_size)],
            'right': [[i, self.env_size - 1] for i in range(self.env_size)]
        }

        self.carn_loc = np.full((self.env_size, self.env_size), None)
        self.herb_loc = np.full((self.env_size, self.env_size), None)

        for carn in self.carns:
            selected_edge = np.random.choice(list(edges.keys()))
            selected_point = edges[selected_edge][np.random.randint(len(edges[selected_edge]))]
            carn.x = selected_point[0]
            carn.y = selected_point[1]
            self.carn_loc[carn.x, carn.y] = carn

        for herb in self.herbs:
            selected_edge = np.random.choice(list(edges.keys()))
            selected_point = edges[selected_edge][np.random.randint(len(edges[selected_edge]))]
            herb.x = selected_point[0]
            herb.y = selected_point[1]
            self.herb_loc[herb.x, herb.y] = herb

    def simulateMatch(self, p1, p2):
        if not p1 and not p2:
            return
        elif p1 and not p2:
            if not p1.isCarn and self.environment[p1.x, p1.y] > 0:
                p1.health += 20
        elif p2 and not p1 :
            if not p2.isCarn and self.environment[p2.x, p2.y] > 0:
                p2.health += 20
        elif p1.isCarn and p2.isCarn:
            # both carnivores are hostile
            p1.health += self.payoff_matrix[0, 0, 0]
            p2.health += self.payoff_matrix[0, 0, 1]
        elif p1.isCarn and not p2.isCarn:
            # p1 will attack p2 and gain health for eating it and p2 gains health if there is food here
            p1.health += self.payoff_matrix[0, 1, 0]
            p2.health += self.payoff_matrix[0, 1, 1] + (40 if self.environment[p2.x, p2.y] > 0 else 0)
        elif not p1.isCarn and p2.isCarn:
            # reverse of above
            p1.health += self.payoff_matrix[1, 0, 0] + (40 if self.environment[p1.x, p1.y] > 0 else 0)
            p2.health += self.payoff_matrix[1, 0, 1]
        elif not p1.isCarn and not p2.isCarn:
            p1.health += self.payoff_matrix[1, 1, 0] + (40 if self.environment[p1.x, p1.y] > 0 else 0)
            p2.health += self.payoff_matrix[1, 1, 1] + (40 if self.environment[p1.x, p1.y] > 0 else 0)

    def runSimulation(self, ticks, daily_moves):
        pygame.init()

        # Set up the display
        screen_size = (1000, 1000)  # Increase the window size
        screen = pygame.display.set_mode(screen_size)
        pygame.display.set_caption('Simulation')

        clock = pygame.time.Clock()

        for tick in range(ticks):
            # at the start of each day spawn food particles in random squares
            total_squares = self.env_size ** 2
            num_food_squares = int(total_squares * self.food_density)
            food_indices = np.random.choice(total_squares, num_food_squares, replace=False)
            for index in food_indices:
                row, col = divmod(index, self.env_size)
                self.environment[row, col] = 1

            # allow each player to make random moves
            for move in range(daily_moves):
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        return

                for player in self.herbs:
                    self.herb_loc[player.x, player.y] = None
                    region = self.carn_loc[max(0, player.x - player.speed):min(self.env_size, player.x + player.speed + 1),
                                           max(0, player.y - player.speed):min(self.env_size, player.y + player.speed + 1)]
                    region_mask = (region == None)
                    if np.any(region_mask):
                        not_carn_indices = np.argwhere(region_mask)
                        present_not_carn = not_carn_indices[np.random.choice(not_carn_indices.shape[0])]
                        dx, dy = present_not_carn[0] - player.speed, present_not_carn[1] - player.speed
                        player.x = (player.x + dx) % self.env_size
                        player.y = (player.y + dy) % self.env_size
                    else:
                        # Move randomly
                        player.x = (player.x + np.random.randint(-player.speed, player.speed + 1)) % self.env_size
                        player.y = (player.y + np.random.randint(-player.speed, player.speed + 1)) % self.env_size

                    self.herb_loc[player.x, player.y] = player

                for player in self.carns:
                    self.carn_loc[player.x, player.y] = None
                    region = self.herb_loc[max(0, player.x - player.speed):min(self.env_size, player.x + player.speed + 1),
                                           max(0, player.y - player.speed):min(self.env_size, player.y + player.speed + 1)]
                    if np.any(region):
                        herb_indices = np.argwhere(region)
                        present_herb = herb_indices[np.random.choice(herb_indices.shape[0])]
                        dx, dy = present_herb[0] - player.speed, present_herb[1] - player.speed
                        player.x = (player.x + dx) % self.env_size
                        player.y = (player.y + dy) % self.env_size
                    else:
                        # Move randomly
                        player.x = (player.x + np.random.randint(-player.speed, player.speed + 1)) % self.env_size
                        player.y = (player.y + np.random.randint(-player.speed, player.speed + 1)) % self.env_size

                    self.carn_loc[player.x, player.y] = player

                # Draw everything
                screen.fill(WHITE)

                for carn in self.carns:
                    pygame.draw.circle(screen, RED, (carn.y * 4, carn.x * 4), 4)  # Scale positions and size
                for herb in self.herbs:
                    pygame.draw.circle(screen, GREEN, (herb.y * 4, herb.x * 4), 4)  # Scale positions and size
                food_indices = np.argwhere(self.environment == 1)
                for row, col in food_indices:
                    pygame.draw.rect(screen, BLUE, pygame.Rect(col * 4, row * 4, 4, 4))  # Scale positions and size

                pygame.display.flip()

                clock.tick(3000)  # Limit to 30 frames per second

            # Process matches and update health scores
            for i in range(self.env_size):
                for j in range(self.env_size):
                    self.simulateMatch(self.carn_loc[i, j], self.herb_loc[i, j])

            # Clear food
            self.environment.fill(0)

            # Remove dead carnivores
            for carn in self.carns:
                if carn.health < 30:
                    if random.randint(1, 100) < 100 - carn.health:
                        self.carn_loc[carn.x, carn.y] = None
                        self.herbs.remove(carn)
                        self.N_carns -= 1
                        self.N -= 1
                        del carn
                    else:
                        # reset this health at the end of the day
                        carn.health-=5
            # Remove dead herbivores
            for herb in self.herbs:
                if herb.health < 30:
                    if random.randint(1, 100) < 100 - herb.health:
                        self.herb_loc[herb.x, herb.y] = None
                        self.herbs.remove(herb)
                        self.N_herbs -= 1
                        self.N -= 1
                        del herb
                    else:
                        # reset this health at the end of the day
                        herb.health-=5
            
            # Reproduce logic can be added here if needed

        pygame.quit()

In [101]:
payoff = np.array([[[-50,-50], [20,-50]],
                  [[-50,20], [0,0]]])
s = Simulation(200,10,100, 300, 0.1, payoff)
s.spawnPlayers()
s.resetPlayerPositions()
s.runSimulation(50, 10)
print(s.N_carns)
print(s.N_herbs)

10
50
